In [1]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
print(f'the array till 5 is {X_train[:5]}')

the array till 5 is [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]


In [4]:
print(f'Shape of X_train : {X_train.shape}')
print(f'Shape of y_train : {y_train.shape}')

Shape of X_train : (10, 3)
Shape of y_train : (10,)


In [5]:
# defining functions 
#  numpy array (y) that indicates whether the examples in that node are edible (1) or poisonous(0) 
def compute_entropy(y):
    entropy=0
    if len(y)!=0:
        p1=p1=len(y[y==1])/len(y)    # p1 ,  which is the fraction of examples that are edible (i.e. have value = 1 in y)# our code goes like p1 is equal to length of y where y==1 divided by total length
        if p1!=0 and p1!=1:
            entropy=-p1*np.log2(p1)-(1-p1)*np.log2(1-p1)
        else:
            entropy=0
    return entropy      

In [6]:
# let's test function with data
# as at root node we have 5 edible and 5 non-edible mushroom therefore entropy=1
# remember u can always dry run #smile
print("Entropy at root node ", compute_entropy(y_train))

Entropy at root node  1.0


In [7]:
# splitting the dataset at the node, segrating where that feature is 1 and 0 into left and right branches respectively
# #e.g For example, say we're starting at the root node (so node_indices = [0,1,2,3,4,5,6,7,8,9]), and we chose to split on feature 0, which is whether or not the example has a brown cap.
# The output of the function is then, left_indices = [0,1,2,3,4,7,9] and right_indices = [5,6,8]

def split_dataset(X,node_indices,feature):
    left_indices=[]
    right_indices=[]


    for i in node_indices:
        if X[i][feature]==1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices,right_indices

In [8]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  # as wi have 10 examples
# let's do it for feature 0
feature=0
left_indices,right_indices=split_dataset(X_train,root_indices,feature)

print("Left root indices ",left_indices)
print("Right root indices ",right_indices)

Left root indices  [0, 1, 2, 3, 4, 7, 9]
Right root indices  [5, 6, 8]


In [9]:
#writing function for the information gain
def compute_information_Gain(X,y,node_indices,feature):
    #splitting the node
    left_indices,right_indices= split_dataset(X,node_indices,feature)
    #segragating the nodes in the basis of left and right node respectively
    X_node,Y_node=X[node_indices],y[node_indices]
    x_left,y_left=X[left_indices],y[left_indices]
    x_right,y_right=X[right_indices],y[right_indices]

    IG=0
    # compute entropy of parent node and the childs left and right
    node_entropy=compute_entropy(y)
    left_entropy=compute_entropy(y_left)
    right_entropy=compute_entropy(y_right)
    # wleft and wright are the proportion of examples at the left and right branch, respectively
    w_left=len(x_left)/len(X)
    w_right=len(x_right)/len(X)
    # weighted entropy
    weighted_entropy=w_left*left_entropy+ w_right*right_entropy
    # Information gain

    IG=node_entropy-weighted_entropy

    return IG

In [10]:
info_gain0= compute_information_Gain(X_train,y_train,root_indices,feature=0)
print("IG at Brown Cap: ",info_gain0)
info_gain1=compute_information_Gain(X_train,y_train,root_indices,feature=1)
print("IG at Stalk Shape: ",info_gain1)
info_gain2=compute_information_Gain(X_train,y_train,root_indices,feature=2)
print("IG at Solidatory: ",info_gain2)

IG at Brown Cap:  0.034851554559677034
IG at Stalk Shape:  0.12451124978365313
IG at Solidatory:  0.2780719051126377


In [11]:
# fn to get dynamically the best decision according to the Information Gain
def get_best_split(X,y,node_indices):
    best_feature=-1
    max_info_gain=0
    n=X.shape[1]
    for ft in range(n):
        info_gain= compute_information_Gain(X,y,node_indices,ft)
        if info_gain>max_info_gain:
            max_info_gain=info_gain
            best_feature=ft
    return best_feature
        

In [12]:
best_feature=get_best_split(X_train,y_train,root_indices)
print(f"Best feature to split: {best_feature}")

Best feature to split: 2


In [13]:
#  building some sort of tree like structure to print our Decision Tree
tree=[]
def DT(X,y,node_indices,branch_name,max_depth,curr_depth):
    if(curr_depth==max_depth):
        formatting=" "*curr_depth+ "-"*curr_depth
        print(formatting,"%s leaf node with Indices" % branch_name, node_indices)
        return
    best_feature=get_best_split(X,y,node_indices)
    tree.append((curr_depth,branch_name,best_feature,node_indices))
    formatting="-"*curr_depth
    print("%s Depth %d , %s: Split on Feature %d"%(formatting,curr_depth,branch_name,best_feature))
    left_indices, right_indices=split_dataset(X,node_indices,best_feature)
    DT(X,y,left_indices,"Left",max_depth,curr_depth+1)
    DT(X,y,right_indices,"Right",max_depth,curr_depth+1)
    

In [14]:
DT(X_train,y_train,root_indices,"Root",2,0)

 Depth 0 , Root: Split on Feature 2
- Depth 1 , Left: Split on Feature 0
  -- Left leaf node with Indices [0, 1, 4, 7]
  -- Right leaf node with Indices [5]
- Depth 1 , Right: Split on Feature 1
  -- Left leaf node with Indices [8]
  -- Right leaf node with Indices [2, 3, 6, 9]
